# Crimes in Europe and Correlation with Social Conditions

## 1. Context

<p>The following analysis aim is to explore European statistics on crime and social conditions to try and find, with a certain degree of reliability, the factors that influence the number of crimes.</p>

This first step is to retrieve, merge, check and clean multiple input data (Eurostat) to output a dataset containing: 
 - The number of crimes per 100k inhabitants by Year and Country as the main datapoints
 - Unemployment rates as a social indicator
 - Early leaving from school as a social indicator

The following data import pipeline is intended as a framework to produce the dataset on which the actual analysis will be carried out, this means that with the same methodology, other social indicators (retrieved from the multitude of tables available on the Eurostat database) can be included.

## 2. Data Source

The only data source is the <a href="https://ec.europa.eu/eurostat/web/main/data/database">Eurostat Database</a>, the statistical office of the European Union providing high-quality statistics in collaboration with National Statistical Institutes and other national authorities in the EU Member States.

The datasets retrieved from the database are:

<ul>
    <li><b><a href="https://ec.europa.eu/eurostat/databrowser/view/crim_gen/default/table?lang=en">estat_crim_gen.csv</a></b> --> Crimes recorded by the police by by offence category</li>
    <li><b><a href="https://ec.europa.eu/eurostat/databrowser/view/demo_gind/default/table?lang=en">estat_demo_gind.csv</a></b> --> Demographic balance and crude rates at national level</li>
  
  <li><b><a href="https://ec.europa.eu/eurostat/databrowser/view/edat_lfse_14/default/table?lang=en">estat_early_leavers_14.csv</a></b> --> Percentage of the population aged 18 to 24 having attained at most lower secondary education and not being involved in further education or training.</li>
    <li><b><a href="https://ec.europa.eu/eurostat/databrowser/view/une_rt_a/default/table?lang=en">estat_une_rt_a.csv</a></b> --> Unemployment as a percentage on the active population</li>
    <li><b>code_transc</b> --> The categorical data in the file is coded as a non intelligible string (e.g. "NR,IT" for Italy), Eurostat provides a set of csv files containing the transcodification from code to labels</li>
</ul>

The datasets contain data split by year and country. The data acquisition step will align and merge using as the main table the <b>estat_crim_gen.csv</b> dataset with the criminality data.  

All the files are located in the folder "dataset_input"

## 3. Data Acquisition

In this step the data is:
    <ul>
        <li> Retrieved from the csv file</li>
        <li> The columns are transcoded using the Eurostat code-label transcodification files</li>
        <li> Filtered, some not useful columns are dropped and the columns are renamed</li>
    </ul>
    
Some utility functions are created to facilitate the steps.

### 3.1 Utilities

The transcoding files are imported and the relationship between column name and transcoding file is set.
The two utility functions below are created to automatically transcode a data frame in place.

In [1]:
import pandas as pd
import numpy as np

# Import of the transcoding files, those have been casted to dictionary after import for performance reasons
# the lookup of a dictionary is much faster than the lookup on the original dataframe produced by pd.read_csv (minutes vs < 3 seconds)
freq_transc = pd.read_csv("dataset_input/code_transc/FREQ_en.csv", header=None, skiprows=[0], index_col = 0, names=['Code', 'Desc']).to_dict()['Desc']
geo_transc = pd.read_csv("dataset_input/code_transc/GEO_en.csv", header=None, skiprows=[0], index_col = 0, names=['Code', 'Desc']).to_dict()['Desc']
iccs_transc = pd.read_csv("dataset_input/code_transc/ICCS_en.csv", header=None, skiprows=[0], index_col = 0, names=['Code', 'Desc']).to_dict()['Desc']
obs_flag_transc = pd.read_csv("dataset_input/code_transc/OBS_FLAG_en.csv", header=None, skiprows=[0], index_col = 0, names=['Code', 'Desc']).to_dict()['Desc']
unit_transc = pd.read_csv("dataset_input/code_transc/UNIT_en.csv", header=None, skiprows=[0], index_col = 0, names=['Code', 'Desc']).to_dict()['Desc']
indic_de_transc = pd.read_csv("dataset_input/code_transc/INDIC_DE_en.csv", header=None, index_col = 0, skiprows=[0], names=['Code', 'Desc']).to_dict()['Desc']
wstatus_transc = pd.read_csv("dataset_input/code_transc/WSTATUS_en.csv", header=None, index_col = 0, skiprows=[0], names=['Code', 'Desc']).to_dict()['Desc']

#Manual adjustment of obs flag found in the datasets web page footnotes but not in transcoding files
obs_flag_transc['b'] = 'break in time series'

#Column name transcoder map
trans_map = {
    'freq': freq_transc,
    'geo': geo_transc,
    'iccs': iccs_transc,
    'OBS_FLAG': obs_flag_transc,
    'unit': unit_transc,
    'indic_de': indic_de_transc,
    'wstatus': wstatus_transc
}

# Returns a label from a code
def transc(item, trans):
    try:
        return trans[item]
    except:
        return item

# Transforms all the encoded columns of a dataframe
def transcode(df):
    for col in df.keys():
        if col in trans_map.keys():
            df[col] = df[col].transform(transc, trans=trans_map[col])

### 3.2 - Number of crimes reported (estat_crim_gen.csv)

The estat_crim_gen.csv file contains the following columns: 

  <ul>
    <li>DATAFLOW --> Name of the flow, always the same value - REMOVED</li>
    <li>freq --> Frequency of the observation, always equals to "Annual" - REMOVED</li>
    <li>unit --> Meaning of the value observed, always equals to "Number" - REMOVED</li>
    <li>OBS_FLAG --> Notes about the observation, always NaN - REMOVED </li>
    <li>iccs --> Typology of Crime - RENAMED</li>
    <li>geo --> Country of reference - RENAMED</li>
    <li>TIME_PERIOD --> Year of the observation - RENAMED</li>
    <li>OBS_VALUE --> The value of the observation - RENAMED</li>
  </ul>

No filtering is required for this dataset.

In [2]:
# Import csv as DataFrame
rep_crimes_raw = pd.read_csv('dataset_input/estat_crim_gen.csv')

# Transcodification of the columns
transcode(rep_crimes_raw)

#Removal of not useful columns and renaming
rep_crimes_t1 = rep_crimes_raw.copy()
rep_crimes_t1.drop(labels=['DATAFLOW', 'freq', 'unit', 'OBS_FLAG'], axis=1, inplace=True)
rep_crimes_t1.rename(columns={'iccs':'Crime Typology', 'geo':'Country', 'TIME_PERIOD':'Year', 'OBS_VALUE': 'Crimes Reported'}, inplace = True)

rep_crimes_t1.head()

,Crime Typology,Country,Year,Crimes Reported
0,Intentional homicide,Austria,1994,88
1,Intentional homicide,Austria,1995,78
2,Intentional homicide,Austria,1996,99
3,Intentional homicide,Austria,1997,66
4,Intentional homicide,Austria,1998,77


### 3.3 - Population on the 1st of January (estat_demo_gind.csv)

The estat_demo_gind.csv file contains the following columns: 

  <ul>
    <li>DATAFLOW --> Name of the flow, always the same value - REMOVED</li>
    <li>freq --> Frequency of the observation, always equals to "Annual" - REMOVED</li>
    <li>OBS_FLAG --> Notes about the observation - RENAMED </li>
    <li>indic_de --> Type of observation, always equal to "Pop on 1 January - total" - REMOVED </li>
    <li>geo --> Country of reference - RENAMED</li>
    <li>TIME_PERIOD --> Year of the observation - RENAMED</li>
    <li>OBS_VALUE --> The value of the observation - RENAMED</li>
  </ul>

The dataset has been filtered to only keep the observation about the <b>Population on 1 January - total</b>.

In [3]:
# Import csv as DataFrame
demo_gind_raw = pd.read_csv('dataset_input/estat_demo_gind.csv')

# Transcodification of the columns
transcode(demo_gind_raw)
demo_gind_t1 = demo_gind_raw.copy()

#Filtering to only keep the population value at 1st of January
demo_gind_t1 = demo_gind_t1[demo_gind_t1['indic_de'] == 'Population on 1 January - total']

#Removal of not useful columns and renaming
demo_gind_t1.drop(labels=['DATAFLOW', 'freq', 'indic_de'], axis=1, inplace=True)
demo_gind_t1.rename(columns={'OBS_FLAG': 'Note','geo':'Country', 'TIME_PERIOD':'Year', 'OBS_VALUE': 'Pop on 1st Jan'}, inplace = True)

demo_gind_t1.head()

,Country,Year,Pop on 1st Jan,Note
33545,Andorra,1986,44593.0,NaN
33546,Andorra,1987,46976.0,NaN
33547,Andorra,1988,48451.0,NaN
33548,Andorra,1989,50528.0,NaN
33549,Andorra,1990,50887.0,NaN


In the estat_demo_gind.csv we have some of the data flagged to indicate that the value has some particularity, the flag that we have on the file have the following meaning (https://stats.oecd.org/glossary/index.htm):

   <ul>
    <li>break in time series --> Breaks in statistical time series occur when there is a change in the standards for defining and observing a variable over time</li>
    <li>provisional --> Data can be subject to review and updates</li>
    <li>estimated --> Value is an estimation of the actual data </li>
   </ul>
   
In this work, given that the flagged elements are a small percentage of the observations, and that the data is coherent between breaks, the flags will be ignored and the column will be removed.

In [4]:
demo_gind_t1.groupby('Note').count()

,Country,Year,Pop on 1st Jan
Note,,,
break in time series,97,97,97
"break in time series, provisional",7,7,7
estimated,62,62,62
"estimated, provisional",4,4,4
provisional,14,14,14


In [5]:
demo_gind_t1.drop(labels=['Note'], axis=1, inplace=True)
demo_gind_t1.head()

,Country,Year,Pop on 1st Jan
33545,Andorra,1986,44593.0
33546,Andorra,1987,46976.0
33547,Andorra,1988,48451.0
33548,Andorra,1989,50528.0
33549,Andorra,1990,50887.0


### 3.4 - Unemployment Percentage Over Active Population (estat_une_rt_a.csv)

The estat_une_rt_a.csv file contains the following columns: 

  <ul>
    <li>DATAFLOW --> Name of the flow, always the same value - REMOVED</li>
    <li>freq --> Frequency of the observation, always equals to "Annual" - REMOVED</li>
    <li>unit --> Meaning of the value observed, data filtered to "Percentage of active population" - REMOVED</li>
    <li>sex --> Sex, data filtered to "T"(Total) - REMOVED </li>
    <li>age --> Age, data filtered to "Y15-75" - REMOVED
    <li>OBS_FLAG --> Notes about the observation - RENAMED </li>
    <li>geo --> Country of reference - RENAMED</li>
    <li>TIME_PERIOD --> Year of the observation - RENAMED</li>
    <li>OBS_VALUE --> The value of the observation - RENAMED</li>
  </ul>

The dataset has been filtered to only keep the values of the total male and female population of age between 15 and 75. The unit of measure is the percentage of the active population.

In [6]:
# Import csv as DataFrame
unemployment_raw = pd.read_csv('dataset_input/estat_une_rt_a.csv')

# Transcodification of the columns
transcode(unemployment_raw)
unemployment_t1 = unemployment_raw.copy()

# Filtering useful data
unemployment_t1 = unemployment_t1[(unemployment_t1['unit'] == 'Percentage of active population') & 
                 (unemployment_t1['sex'] == 'T') &
                 (unemployment_t1['age'] == 'Y15-74')]

#Removal of not useful columns and renaming
unemployment_t1.drop(labels=['DATAFLOW', 'freq', 'unit','sex','age'], axis=1, inplace=True)
unemployment_t1.rename(columns={'OBS_FLAG': 'Note','geo':'Country', 'TIME_PERIOD':'Year', 'OBS_VALUE': 'Unemp Perc'}, inplace = True)

unemployment_t1.head()

,Country,Year,Unemp Perc,Note
10142,Austria,1995,4.2,estimated
10143,Austria,1996,4.6,estimated
10144,Austria,1997,4.6,estimated
10145,Austria,1998,4.4,estimated
10146,Austria,1999,3.7,NaN


In [7]:
unemployment_t1.groupby('Note').count()

,Country,Year,Unemp Perc
Note,,,
break in time series,61,61,61
"break in time series, provisional",1,1,1
estimated,152,152,152


In [8]:
unemployment_t1.drop(labels=['Note'], axis=1, inplace=True)
unemployment_t1.head()

,Country,Year,Unemp Perc
10142,Austria,1995,4.2
10143,Austria,1996,4.6
10144,Austria,1997,4.6
10145,Austria,1998,4.4
10146,Austria,1999,3.7


### 3.4 - Early Leavers From School (estat_une_rt_a.csv)

The estat_une_rt_a.csv file contains the following columns: 

  <ul>
    <li>DATAFLOW --> Name of the flow, always the same value - REMOVED</li>
    <li>freq --> Frequency of the observation, always equals to "Annual" - REMOVED</li>
    <li>unit --> Meaning of the value observed, always equal to "Percentage" - REMOVED</li>
    <li>sex --> Sex, data filtered to "T"(Total) - REMOVED </li>
    <li>age --> Age, always equal to "Y18-24" - REMOVED
    <li>OBS_FLAG --> Notes about the observation - RENAMED </li>
    <li>geo --> Country of reference - RENAMED</li>
    <li>TIME_PERIOD --> Year of the observation - RENAMED</li>
    <li>OBS_VALUE --> The value of the observation - RENAMED</li>
  </ul>

The dataset has been filtered to only keep the values of the total of male and female population, and the total of the population regardless the status (e.g. Employed, Unemployed...).
 

In [9]:
early_leavers_raw = pd.read_csv('dataset_input/estat_early_leavers_14.csv')

transcode(early_leavers_raw)
early_leavers_t1 = early_leavers_raw.copy()
early_leavers_t1 = early_leavers_t1[(early_leavers_t1['wstatus'] == 'Population') & 
                 (early_leavers_t1['sex'] == 'T')]


early_leavers_t1.drop(labels=['DATAFLOW', 'freq', 'unit','sex','age', 'wstatus'], axis=1, inplace=True)
early_leavers_t1.rename(columns={'OBS_FLAG': 'Note','geo':'Country', 'TIME_PERIOD':'Year', 'OBS_VALUE': 'Early Leavers Perc'}, inplace = True)

early_leavers_t1.head()

,Country,Year,Early Leavers Perc,Note
8414,Austria,1995,13.6,NaN
8415,Austria,1996,12.1,NaN
8416,Austria,1997,10.8,NaN
8417,Austria,1999,10.7,NaN
8418,Austria,2000,10.2,NaN


In this file we have some other flags :
 <ul>
    <li> definition differs --> different definition of the value observed, looking at the metadata those seem to be related to different interpretation of "lower secondary education" in the years</li>
    <li>low reliability --> Low guarantee that the data reflects the reality
 </ul>

Again the flagged values are a small percentage of the dataset and the value are close enough (no statistical fluctuation) to permit the ignoring of the column.

In [10]:
early_leavers_t1.groupby('Note').count()

,Country,Year,Early Leavers Perc
Note,,,
break in time series,162,162,162
"break in time series, low reliability",3,3,3
"break in time series, provisional",1,1,1
definition differs (see metadata),5,5,5
low reliability,7,7,7


In [11]:
early_leavers_t1.drop(labels=['Note'], axis=1, inplace=True)
early_leavers_t1.head()

,Country,Year,Early Leavers Perc
8414,Austria,1995,13.6
8415,Austria,1996,12.1
8416,Austria,1997,10.8
8417,Austria,1999,10.7
8418,Austria,2000,10.2


## 4. Data Alignment

Now that the input datasets are filtered and cleaned the next step is to ensure that the data is aligned and can be merged in one dataset.
The merging of the datasets will be executed using Country and Year as key, so this step aims to make sure that all the Years and the Countries that are in the main dataset (crimes report) are also in the other ones.

For the years everything is already fine, the crime report has from 1993 to 2007, and all the other datasets have at least those years. 
For the countries, we have some misalignment to take care of.

1. For the population dataset we have the United Kingdom as a single country, while in the criminality dataset we have the United Kingdom states split among Scotland, England and Wales, Northern Ireland.

2. For the unemployment dataset we have the same problem of the population dataset but also Liechtenstein is missing

3. For the early leaving from school dataset we have the same problem of the above, but also France (metropolitan) is missing (that is France plus Corsica without colonial territories)

In [12]:
#Using set difference operator to establish missing countries
missing_countries_pop = set(rep_crimes_t1['Country']) - set(demo_gind_t1['Country'])
missing_countries_unemp = set(rep_crimes_t1['Country']) - set(unemployment_t1['Country'])
missing_countries_early = set(rep_crimes_t1['Country']) - set(early_leavers_t1['Country'])

print(missing_countries_pop)
print(missing_countries_unemp)
print(missing_countries_early)

{'England and Wales', 'Scotland', 'Northern Ireland (UK)'}
{'England and Wales', 'Scotland', 'Northern Ireland (UK)', 'Liechtenstein'}
{'Scotland', 'Liechtenstein', 'England and Wales', 'France (metropolitan)', 'Northern Ireland (UK)'}


The problem of the United Kingdom will be resolved by aggregating the single state data (Scotland, England and Wales, Northern Ireland) into a United Kingdom data series that will be appended to the crimes report dataset. After that, the single state rows will be removed.

In [13]:
#Creating a mask to get the missing countries rows
remove_missing = lambda x: any(country for country in missing_countries_pop if country in x)
mask = rep_crimes_t1.Country.apply(remove_missing)

#Aggregating the data to generate the United Kingdom values (sum of the number of crimes reported)
grouped_missing = rep_crimes_t1[mask].groupby(['Year','Crime Typology']).sum().reset_index()
grouped_missing['Country'] = 'United Kingdom'


#Adding to the dataset the new rows of the United Kingdom
rep_crimes_t2 = rep_crimes_t1.copy()
rep_crimes_t2 = rep_crimes_t2.append(grouped_missing, sort = True, ignore_index = True)

#removal of the single states rows
mask = rep_crimes_t2.Country.apply(remove_missing)
rep_crimes_t2 = rep_crimes_t2[~mask]

For Liechtenstein and France(metropolitan) the data is simply missing, so the only option is to remove those two countries.

This can be safely done because we still have France as a country and the metropolitan definition of the country France (metropolitan) is not essential for the analysis.

Liechtenstein is a very small country (40k inhabitants), almost a city, the statistical value of it is almost zero.

In [14]:
#Removing country with social indicators not present
rep_crimes_t3 = rep_crimes_t2.copy()
rep_crimes_t3 = rep_crimes_t3[~(rep_crimes_t3['Country'] == 'Liechtenstein')]
rep_crimes_t4 = rep_crimes_t3.copy()
rep_crimes_t4 = rep_crimes_t4[~(rep_crimes_t4['Country'] == 'France (metropolitan)')]

In [15]:
missing_countries_pop = set(rep_crimes_t4['Country']) - set(demo_gind_t1['Country'])
missing_countries_unemp = set(rep_crimes_t4['Country']) - set(unemployment_t1['Country'])
missing_countries_early = set(rep_crimes_t4['Country']) - set(early_leavers_t1['Country'])

print(missing_countries_pop)
print(missing_countries_unemp)
print(missing_countries_early)

set()
set()
set()


## 5. Data Merging

Below the four datasets are merged with a left join from the crime report dataset.
One more column is added, the "Crimes per 100k" which represents the number of crimes per 100000 inhabitants, this will be the target value to evaluate criminality between countries with a different demography.

In [16]:
rep_crimes_t5 = rep_crimes_t4.copy()

rep_crimes_t5 = pd.merge(rep_crimes_t5,demo_gind_t1, on=['Country', 'Year'], how='left' )
rep_crimes_t5['Crimes per 100k'] = (rep_crimes_t5['Crimes Reported']/(rep_crimes_t5['Pop on 1st Jan']/100000))
rep_crimes_t5 = pd.merge(rep_crimes_t5,unemployment_t1, on=['Country', 'Year'], how='left' )
rep_crimes_t5 = pd.merge(rep_crimes_t5,early_leavers_t1, on=['Country', 'Year'], how='left' )

rep_crimes_t5.head()

,Country,Crime Typology,Crimes Reported,Year,Pop on 1st Jan,Crimes per 100k,Unemp Perc,Early Leavers Perc
0,Austria,Intentional homicide,88,1994,7928746.0,1.109885,NaN,NaN
1,Austria,Intentional homicide,78,1995,7943489.0,0.981936,4.2,13.6
2,Austria,Intentional homicide,99,1996,7953067.0,1.244803,4.6,12.1
3,Austria,Intentional homicide,66,1997,7964966.0,0.828629,4.6,10.8
4,Austria,Intentional homicide,77,1998,7971116.0,0.965988,4.4,NaN


## 6. Missing Values

Now that the final dataset is in one piece, let's check the missing values and the missing data points.

Below we can see that the social indicators have very high percentage of missing values

In [17]:
# Concatenating two dataseries in one dataframe to have the different aggregations needed
summary = pd.concat([rep_crimes_t5.isna().sum(), rep_crimes_t5.count()], axis=1, keys=['Missing Values', 'Total Values'])
# Adding percentage of missing
summary['Perc Missing'] = (summary['Missing Values'] / summary['Total Values'])*100

summary

,Missing Values,Total Values,Perc Missing
Country,0,3146,0.000000
Crime Typology,0,3146,0.000000
Crimes Reported,0,3146,0.000000
Year,0,3146,0.000000
Pop on 1st Jan,0,3146,0.000000
Crimes per 100k,0,3146,0.000000
Unemp Perc,758,2388,31.742044
Early Leavers Perc,996,2150,46.325581


Here we define a summary view to check in more details where the missing values are coming from.

Looking closely at the data two considerations can be made: 

  1. The smallest countries are the ones with the most missing data
  2. The older are the observations the more the data is missing

In [18]:
# Function to create a report of the missing data, can be used to aggregate by country or year and the result
# will be a percentage of the missing values over the total
# Only countries or year with missing values percentage != 0 are extracted
def get_summary(rep_crimes, col):
    summary = rep_crimes.copy()
    #Masking the social indicators columns
    summary['Unemp Perc'] = summary['Unemp Perc'].isna()
    summary['Early Leavers Perc'] = summary['Early Leavers Perc'].isna()
    
    #Aggregating the values needed (aggregation is on the parameter column)
    summary = summary[[col,'Pop on 1st Jan','Early Leavers Perc','Unemp Perc']].groupby(col).agg(
        mean_population = ('Pop on 1st Jan', 'mean'),
        nan_early_leavers = ('Early Leavers Perc', 'sum'),
        nan_unemp_perc = ('Unemp Perc', 'sum'),
        total = ('Early Leavers Perc', 'count')
    ).reset_index()
    
    #Renaming the columns
    summary.rename(columns={'mean_population':'Mean Pop','nan_early_leavers':'NaN Early Leavers', 'nan_unemp_perc': 'NaN Unemp Perc', 'total': 'Total'}, inplace=True)
    
    #Filtering rows with 0 missing values
    summary = summary[(summary['NaN Early Leavers'] != 0) | (summary['NaN Unemp Perc'] != 0)]
    
    #Calculating pergentages
    summary['Nan Early Leavers Perc'] = (summary['NaN Early Leavers']/summary['Total'])*100
    summary['Nan Unemp Perc'] = (summary['NaN Unemp Perc']/summary['Total'])*100

    return summary

# summary = get_summary(rep_crimes_t5,'Year') --> to have a report on the year
summary = get_summary(rep_crimes_t5,'Country')
summary


,Country,Mean Pop,NaN Early Leavers,NaN Unemp Perc,Total,Nan Early Leavers Perc,Nan Unemp Perc
0,Austria,8.068799e+06,10,4,89,11.235955,4.494382
2,Bulgaria,7.992132e+06,34,27,83,40.963855,32.530120
3,Croatia,4.401335e+06,37,23,79,46.835443,29.113924
4,Cyprus,6.895661e+05,42,28,105,40.000000,26.666667
5,Czechia,1.026089e+07,54,28,96,56.250000,29.166667
7,Estonia,1.397334e+06,30,23,98,30.612245,23.469388
8,Finland,5.173085e+06,17,17,100,17.000000,17.000000
9,France,6.102360e+07,0,63,98,0.000000,64.285714
10,Germany (until 1990 former territory of the FRG),8.209802e+07,24,0,101,23.762376,0.000000
12,Hungary,1.022024e+07,28,28,105,26.666667,26.666667


The approach will be to remove the smallest countries and the heuristic rule would be all the countries with a population below the 50th percentile. It still leaves the dataset with the most statistical relevant countries (more populous)

After that, removing the data before 1998 gives a reasonable percentage of missing values for the countries taken singularly. The dataset still contains 10 years of data.

In [19]:
# 50th percentile over the population in the dataset
percentile_50 = np.percentile(rep_crimes_t5['Pop on 1st Jan'],50)

#retrieving a list of the countries to remove 
small_countries = rep_crimes_t5[rep_crimes_t5['Pop on 1st Jan'] < percentile_50]['Country']

# Removing the countries
rep_crimes_t6 = rep_crimes_t5.copy()
rep_crimes_t6 = rep_crimes_t6[~(rep_crimes_t6['Country'].isin(small_countries))]

# Removing the rows regarding years before 1998
rep_crimes_t7 = rep_crimes_t6.copy()
rep_crimes_t7 = rep_crimes_t7[(rep_crimes_t7['Year'] > 1997) & (rep_crimes_t7['Country'] != 'Turkey') ]

summary = get_summary(rep_crimes_t7, 'Country')
summary[['Country','Nan Early Leavers Perc','Nan Unemp Perc']]

,Country,Nan Early Leavers Perc,Nan Unemp Perc
1,Czechia,38.235294,0.0
2,France,0.000000,50.0
3,Germany (until 1990 former territory of the FRG),10.000000,0.0
8,Poland,30.000000,0.0
12,Sweden,10.000000,0.0
13,United Kingdom,10.000000,0.0


The remaining missing values are imputed, the rule for this simple imputation is to replace the missing values with the mean of the country for the value.

In [20]:
rep_crimes_t8 = rep_crimes_t7.copy()

#Using fillna and groupby to impute the mean of the reference country
rep_crimes_t8['Unemp Perc'] = rep_crimes_t8['Unemp Perc'].fillna(rep_crimes_t8.groupby('Country')['Unemp Perc'].transform('mean'))
rep_crimes_t8['Early Leavers Perc'] = rep_crimes_t8['Early Leavers Perc'].fillna(rep_crimes_t8.groupby('Country')['Early Leavers Perc'].transform('mean'))

summary = get_summary(rep_crimes_t8, 'Country')
summary

,Country,Mean Pop,NaN Early Leavers,NaN Unemp Perc,Total,Nan Early Leavers Perc,Nan Unemp Perc


## 7. Critical Analysis

The final dataset for the analysis is composed as follows:

- Country --> 14 countries {Belgium, Czechia, France, Germany, Greece, Hungary, Italy, Netherlands, Poland, Portugal, Romania, Spain, Sweden, United Kingdom}
- Crime Typology -->  6 typologies {Violent Crimes, Burglary, Homicide, Robbery, Vehicle Theft, Drugs}
- Year --> 10 Years, from 1998 to 2007
- Crimes Reported --> Number of crimes reported
- Crimes per 100k --> Number of crimes reported per 100k inhabitants
- Pop on 1st Jan --> Population on the 1st January of the year
- Unemp Perc --> Unemployment percentage over the active population
- Early Leavers Perc --> Percentage of people leaving the school of age between 18 and 24

In [21]:
rep_crimes_t8.head()

,Country,Crime Typology,Crimes Reported,Year,Pop on 1st Jan,Crimes per 100k,Unemp Perc,Early Leavers Perc
14,Belgium,Intentional homicide,216,2000,10239085.0,2.109564,7.0,13.8
15,Belgium,Intentional homicide,282,2001,10263414.0,2.747624,6.6,13.8
16,Belgium,Intentional homicide,320,2002,10309725.0,3.103866,7.5,14.1
17,Belgium,Intentional homicide,229,2003,10355844.0,2.211312,8.2,14.3
18,Belgium,Intentional homicide,267,2004,10396421.0,2.568191,8.4,13.1


Aggregating the data on the Country column we can see that the criminality does not seem to be related to the social indicators. The country with the most criminality is Sweden and Romania has the lowest criminality rate.
Two considerations can be made:

- The crimes reported are a mix of non-violent or minor crimes and capital crimes, all of those have the same weight if we aggregate only at the Country level. The minor crimes are vastly higher in number than the most violent crimes
- Those are the crimes reported to the police, in reality, countries with rough criminalities probably have a lower report rate for minor crimes (e.g. vehicle theft).

In [22]:
rep_crimes_t8[rep_crimes_t8['Crime Typology'] == 'TOTAL'].groupby('Country').mean()[['Crimes per 100k','Unemp Perc','Early Leavers Perc']].sort_values(axis=0, by='Crimes per 100k', ascending=False)

,Crimes per 100k,Unemp Perc,Early Leavers Perc
Country,,,
Sweden,13740.351248,6.46,8.911111
United Kingdom,10098.818610,5.34,15.211111
Belgium,9704.223863,7.75,13.337500
Netherlands,8271.135779,3.94,14.500000
Germany (until 1990 former territory of the FRG),7783.502596,9.40,13.211111
France,6158.082134,8.62,13.280000
Spain,4991.860024,11.87,30.480000
Hungary,4489.153525,6.75,12.910000
Italy,4279.676468,8.98,23.890000


With the assumption that homicide are always reported in all the countries and that is the best indicator of the criminality index, we look at the dataset filtering on this type of crime.

We have the opposite situation, Romania is the country with the most criminality and Sweden at the bottom of the list.

Still, the correlation with the social indicators selected is not obvious, there are a few outliers with bad social indicators but good criminality rate.

In [23]:
rep_crimes_t8[rep_crimes_t8['Crime Typology'] == 'Intentional homicide'].groupby('Country').mean()[['Crimes per 100k','Unemp Perc','Early Leavers Perc']].sort_values(axis=0, by='Crimes per 100k', ascending=False)

,Crimes per 100k,Unemp Perc,Early Leavers Perc
Country,,,
Romania,2.391901,7.22,20.790000
Belgium,2.371402,7.75,13.337500
Hungary,2.080535,6.75,12.910000
Poland,1.752229,15.80,5.985714
United Kingdom,1.668863,5.34,15.211111
France,1.588807,8.62,13.280000
Czechia,1.464774,7.61,5.833333
Italy,1.321751,8.98,23.890000
Portugal,1.291540,5.97,41.830000


Let's look at an outlier, Italy, which has a very high rate of early leavers from school and a pretty high unemployment rate but an average criminality rate.

For this single country seems that the relationship between criminality and social indicators is very strong, the indicators improve with the passing of the years and so does the criminality rate. (The reader can easily explore different countries changing the value of "Italy" below)

In [24]:
rep_crimes_t8[
    (rep_crimes_t8.Country == 'Italy') & 
    (rep_crimes_t8['Crime Typology'] == 'Intentional homicide')].groupby('Year').mean()[['Crimes per 100k','Unemp Perc','Early Leavers Perc']]

,Crimes per 100k,Unemp Perc,Early Leavers Perc
Year,,,
1998,1.613233,11.9,28.4
1999,1.500639,11.4,27.2
2000,1.437016,10.6,25.1
2001,1.343031,9.6,25.9
2002,1.212546,9.0,24.2
2003,1.339039,8.7,23.0
2004,1.334008,8.0,23.1
2005,1.119659,7.7,22.1
2006,1.141839,6.8,20.4


Dataset output can be found in the "dataset_output" folder

In [25]:
rep_crimes_t8.to_csv("dataset_output/rep_crimes.csv")

## 8. Conclusions & Limitations

1. A correlation between the social indicators chosen seem to be present for the single country time series
2. The stronger indicator seems to be the unemployment rate over the percentage of early leavers from school
3. Data between countries of the European community chosen is very hard to compare, the criminality decreases when the social indicators improve, but the countries with the worst indicators are not the ones with the most criminality. This is probably a symptom that to have a better picture more social indicators are needed.
4. Crime typologies are very heterogeneous and of different scales (there are far more robberies than homicides). So is best to focus on one typology at a time, and treat them as different criminality measures, probably for different types of criminality different social indicators are needed

## 9. Further Work

1. Ideas for analysis:

    - Clusterizzation of the countries
    - Linear regression on different data aggregates based on the clusterization on the single country, for the single indicator
    - Correlation index of the social indicators with criminality rates one by one
    - All the above given different weights to the social indicators
    


2. Adding more social indicators with the framework described, interesting indicators can be:
    
    - Immigration rates from poor countries
    - Average income
    - Prison and inmates conditions (social inclusion)
    - Lenght of trials
    
    
3. Adding more years, the criminal rates treated in this work  are the historical ones, another dataset from Eurostat Database contains data from 2007 to 2019, it can be included in the analysis to have more depth 